In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
caffe_root = '/home/jonnatas/workspace_python/caffe/'
sys.path.insert(0, caffe_root + 'python')

import os
import cv2
import caffe


In [2]:
import os
if os.path.isfile(caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'):
    print 'CaffeNet Found !!!!!!'
else:
    print 'Downloading pre-trained CaffeNet model...'
    !../scripts/download_model_binary.py ../models/bvlc_reference_caffenet

CaffeNet Found !!!!!!


In [3]:
caffe.set_mode_cpu()

model_def = caffe_root + 'models/bvlc_reference_caffenet/deploy.prototxt'
model_weights = caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'

net = caffe.Net(model_def,      # defines the structure of the model
                model_weights,  # contains the trained weights
                caffe.TEST)     # use test mode (e.g., don't perform dropout)

In [4]:
# load the mean ImageNet image (as distributed with Caffe) for subtraction
mu = np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy')
mu = mu.mean(1).mean(1)  # average over pixels to obtain the mean (BGR) pixel values
print 'mean-subtracted values:', zip('BGR', mu)

# create transformer for the input called 'data'
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})

transformer.set_transpose('data', (2,0,1))  # move image channels to outermost dimension
transformer.set_mean('data', mu)            # subtract the dataset-mean value in each channel
transformer.set_raw_scale('data', 255)      # rescale from [0, 1] to [0, 255]
transformer.set_channel_swap('data', (2,1,0))  # swap channels from RGB to BGR

mean-subtracted values: [('B', 104.0069879317889), ('G', 116.66876761696767), ('R', 122.6789143406786)]


In [5]:
# set the size of the input (we can skip this if we're happy
#  with the default; we can also change it later, e.g., for different batch sizes)
net.blobs['data'].reshape(50,        # batch size
                          3,         # 3-channel (BGR) images
                          227, 227)  # image size is 227x227

In [6]:
# copy the image data into the memory allocated for the net
net.blobs['data'].data[...] = transformed_image

### perform classification
output = net.forward()

output_prob = output['prob'][0]  # the output probability vector for the first image in the batch

print 'predicted class is:', output_prob.argmax()

NameError: name 'transformed_image' is not defined

In [14]:
# load ImageNet labels
labels_file = caffe_root + 'data/ilsvrc12/synset_words.txt'
if not os.path.exists(labels_file):
    !../data/ilsvrc12/get_ilsvrc_aux.sh
    
labels = np.loadtxt(labels_file, str, delimiter='\t')

print 'output label:', labels[output_prob.argmax()]

output label:

NameError: name 'output_prob' is not defined

In [13]:
# download an image
def imageDowload(path):
    my_image_url = path  # paste your URL here
    # for example:
    #my_image_url = "https://upload.wikimedia.org/wikipedia/commons/b/be/Orang_Utan%2C_Semenggok_Forest_Reserve%2C_Sarawak%2C_Borneo%2C_Malaysia.JPG"
    !wget -O image.jpg $my_image_url

    # transform it and copy it into the net
    image = caffe.io.load_image('image.jpg')
    net.blobs['data'].data[...] = transformer.preprocess('data', image)

    # perform classification
    net.forward()

    # obtain the output probabilities
    output_prob = net.blobs['prob'].data[0]

    # sort top five predictions from softmax output
    top_inds = output_prob.argsort()[::-1][:5]

    plt.imshow(image)

    print 'probabilities and labels:'
    return zip(output_prob[top_inds], labels[top_inds])

imageDowload("http://imguol.com/c/noticias/af/2016/03/28/gata-cupcake-1459205092117_615x300.jpg")

--2016-09-01 20:57:26--  http://imguol.com/c/noticias/af/2016/03/28/gata-cupcake-1459205092117_615x300.jpg
Resolvendo imguol.com (imguol.com)... 2804:49c:3101:405:ffff:ffff:ffff:17, 2804:49c:3103:405:ffff:ffff:ffff:1, 200.221.7.95, ...
Conectando-se a imguol.com (imguol.com)|2804:49c:3101:405:ffff:ffff:ffff:17|:80... conectado.
A requisição HTTP foi enviada, aguardando resposta... 200 OK
Tamanho: 77179 (75K) [image/jpeg]
Salvando em: “image.jpg”

100%[======================================>] 77.179      --.-K/s   em 0,08s   

2016-09-01 20:57:26 (910 KB/s) - “image.jpg” salvo [77179/77179]

probabilities and labels:


NameError: global name 'labels' is not defined